In [ ]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [ ]:
import os
import shutil
import random
PROJ_DIR = os.environ["RNB_PLANNING_DIR"]
os.chdir(os.path.join(PROJ_DIR, "src"))

from pkg.utils.utils_python3 import *
DATA_PATH = os.path.join(PROJ_DIR, "data")
LAT_DATA_PATH = os.path.join(DATA_PATH, "latticized")
MODEL_PATH = os.path.join(PROJ_DIR, "model")
LAT_MODEL_PATH = os.path.join(MODEL_PATH,"latticized_full")
try_mkdir(MODEL_PATH)
try_mkdir(LAT_MODEL_PATH)
GRASP_FOLDER = "grasp"
ARM10_FOLDER = "arm_10"
ARM05_FOLDER = "arm_05"
FULLS_FOLDER = "full_scene"

In [ ]:
ROBOT_TYPE_NAME="indy7"
ROBOT_DATA_ROOT = os.path.join(LAT_DATA_PATH, ROBOT_TYPE_NAME)
# ROBOT_DATA_ROOT = LAT_DATA_PATH
ROBOT_MODEL_ROOT =  os.path.join(LAT_MODEL_PATH, ROBOT_TYPE_NAME)
ARM_FOLDER = ARM10_FOLDER

In [ ]:
dataset_list = sorted(os.listdir(ROBOT_DATA_ROOT))
DATASET_TRAIN = dataset_list[:10]
DATASET_TEST = dataset_list[10:15]
print(DATASET_TRAIN)
print(DATASET_TEST)

In [ ]:
# DATASET_TRAIN = ['20210214-232708', '20210215-041031', '20210215-085110', '20210215-133753', '20210215-184319', 
#                  '20210216-005455', '20210216-054418', '20210216-104554', '20210216-152114', '20210216-201729']
# DATASET_TEST = ['20210217-010926', '20210217-063641', '20210217-113319', '20210217-162106', '20210217-205606']
# DATASET_TRAIN = ['20210219-091338', '20210219-124428', '20210219-234147', '20210220-035639', '20210220-080119', 
#                  '20210220-122304', '20210220-160737', '20210220-194129', '20210220-234400', '20210221-043209']
# DATASET_TEST = ['20210221-082144', '20210221-123619', '20210221-160542', '20210221-195509', '20210221-234239']
FULL_SHAPE = (60,60,60)

dataset_train = []
for dataset in DATASET_TRAIN:
    file_list = sorted(os.listdir(os.path.join(ROBOT_DATA_ROOT, dataset, FULLS_FOLDER)))
    for file in file_list:
        dataset_train.append(os.path.join(ROBOT_DATA_ROOT, dataset, FULLS_FOLDER, file))
print("train set: {}".format(len(dataset_train)))        


dataset_test = []
for dataset in DATASET_TEST:
    file_list = sorted(os.listdir(os.path.join(ROBOT_DATA_ROOT, dataset, FULLS_FOLDER)))
    for file in file_list:
        dataset_test.append(os.path.join(ROBOT_DATA_ROOT, dataset, FULLS_FOLDER, file))
print("train set: {}".format(len(dataset_test)))        

In [ ]:
def load_data(data_path):
    full_data = load_pickle(data_path)
    full_tar_idx = full_data[b'tar']
    full_obj_idx = full_data[b'obj']
    full_tool_idx = full_data[b'tool']
    reach_lb = full_data[b'reach']
    retrieve_lb = full_data[b'retrieve']
    full_tool_img = np.zeros(FULL_SHAPE)
    full_tool_img[np.unravel_index(full_tool_idx, shape=FULL_SHAPE)] = 1
    full_obj_img = np.zeros(FULL_SHAPE)
    full_obj_img[np.unravel_index(full_obj_idx, shape=FULL_SHAPE)] = 1
    full_tar_img = np.zeros(FULL_SHAPE)
    full_tar_img[np.unravel_index(full_tar_idx, shape=FULL_SHAPE)] = 1
    full_img = np.stack([full_tool_img, full_obj_img, full_tar_img], axis=-1)
    label = np.array([reach_lb, retrieve_lb])
    return full_img, label

## Train

In [ ]:
from pkg.planning.filtering.lattice_model.lattice_model_full import *

# Create an instance of the model
model = ResNetModelTP_FULL()

loss_object = tf.keras.losses.BinaryCrossentropy()
optimizer = tf.keras.optimizers.Adam()

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.BinaryAccuracy(name='train_accuracy')
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        # training=True is only needed if there are layers with different
        # behavior during training versus inference (e.g. Dropout).
        predictions = model(images, training=True)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)
    

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.BinaryAccuracy(name='test_accuracy')
@tf.function
def test_step(images, labels):
    # training=False is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(images, training=False)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [ ]:
ROBOT_MODEL_ROOT =  os.path.join(LAT_MODEL_PATH, ROBOT_TYPE_NAME)
current_time = get_now()
logpath = os.path.join(ROBOT_MODEL_ROOT, current_time)
try_mkdir(logpath)
train_log_dir = os.path.join(logpath, 'train')
test_log_dir = os.path.join(logpath, 'test')
model_log_dir = os.path.join(logpath, 'model_{}/')
train_summary_writer = tf.summary.create_file_writer(train_log_dir)
test_summary_writer = tf.summary.create_file_writer(test_log_dir)
shutil.copy(os.path.join(PROJ_DIR,'src', 'pkg','planning','filtering','lattice_model','lattice_model_full.py' ), logpath)
print(f'Log path: {logpath}')

In [ ]:
EPOCHS_S = 0
EPOCHS_E = 20
BATCH_SIZE = 16
LOG_STEP = 100
N_train = len(dataset_train)
N_test = len(dataset_test)

for epoch in range(EPOCHS_S, EPOCHS_E):
    # Reset the metrics at the start of the next epoch
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()
    
    random.shuffle(dataset_train)
    i_step = 0
    data_batch, label_batch = [], []
    for data_path in dataset_train:
        i_step += 1
        full_img, label = load_data(data_path)
        data_batch.append(full_img)
        label_batch.append(label)
        if i_step%BATCH_SIZE==0:
            data_batch = np.array(data_batch)
            label_batch = np.array(label_batch, dtype=np.int)
            train_step(data_batch, label_batch)
            data_batch, label_batch = [], []
        if i_step%LOG_STEP==0:
            print("train step - {}/{}        ".format(i_step, N_train), end = '\r')
    with train_summary_writer.as_default():
        tf.summary.scalar('loss', train_loss.result(), step=epoch)
        tf.summary.scalar('accuracy', train_accuracy.result(), step=epoch)

    i_step = 0
    data_batch, label_batch = [], []
    for data_path in dataset_test:
        i_step += 1
        full_img, label = load_data(data_path)
        data_batch.append(full_img)
        label_batch.append(label)
        if i_step%BATCH_SIZE==0:
            data_batch = np.array(data_batch)
            label_batch = np.array(label_batch, dtype=np.int)
            test_step(data_batch, label_batch)
            data_batch, label_batch = [], []
        if i_step%LOG_STEP==0:
            print("test step - {}/{}        ".format(i_step, N_test), end = '\r')
    with test_summary_writer.as_default():
        tf.summary.scalar('loss', test_loss.result(), step=epoch)
        tf.summary.scalar('accuracy', test_accuracy.result(), step=epoch)
            
    model.save(model_log_dir.format(epoch + 1))

    print("")
    print("=================================================================")
    print(
        f'Epoch {epoch + 1}, '
        f'Loss: {train_loss.result()}, '
        f'Accuracy: {train_accuracy.result() * 100}, '
        f'Test Loss: {test_loss.result()}, '
        f'Test Accuracy: {test_accuracy.result() * 100}'
    )
    print("=================================================================")
    print("")

In [ ]:
INDY7
=================================================================
Epoch 1, Loss: 0.7103465795516968, Accuracy: 50.663936614990234, Test Loss: 0.710077166557312, Test Accuracy: 52.84455490112305
=================================================================

=================================================================
Epoch 4, Loss: 0.3553343713283539, Accuracy: 85.30850219726562, Test Loss: 0.3638063967227936, Test Accuracy: 84.30488586425781
=================================================================

=================================================================
Epoch 9, Loss: 0.21100938320159912, Accuracy: 92.50698852539062, Test Loss: 0.47806230187416077, Test Accuracy: 83.81410217285156
=================================================================

=================================================================
Epoch 10, Loss: 0.17595909535884857, Accuracy: 93.27076721191406, Test Loss: 0.6171865463256836, Test Accuracy: 84.01441955566406
=================================================================

PANDA
=================================================================
Epoch 20, Loss: 0.6762306690216064, Accuracy: 59.23544692993164, Test Loss: 0.8472959995269775, Test Accuracy: 58.97563552856445
=================================================================


## Load & test

In [ ]:
last_model = sorted(os.listdir(ROBOT_MODEL_ROOT))[-1]
logpath = os.path.join(ROBOT_MODEL_ROOT, last_model)

model_epoch_list = []
acc_epoch_list = []
loss_epoch_list = []
last_save = sorted([item for item in os.listdir(logpath) if item.startswith("model")], key=lambda x: int(x[6:]))[-1]
# last_save = 'model_1'
model_log_dir = os.path.join(logpath, last_save)

import tensorflow as tf
model = tf.keras.models.load_model(model_log_dir)

@tf.function
def inference(images):
    # training=False is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(images, training=False)
    return predictions

loss_object = tf.keras.losses.BinaryCrossentropy()

@tf.function
def calc_loss(labels, predictions):
    # training=False is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    return loss_object(labels, predictions)

BATCH_SIZE = 1
LOG_STEP = 100
N_test = len(dataset_test)
gtimer = GlobalTimer.instance()
gtimer.reset()

full_img, label = load_data(dataset_test[0])
res = inference(np.array([full_img]))

i_step = 0
res_list = []
label_list = []
loss_list= []
img_batch = []
label_batch = [] 
for data_path in dataset_test:
    i_step += 1
    full_img, label = load_data(data_path)
    img_batch.append(full_img)
    label_batch.append(label)
    if len(img_batch)==BATCH_SIZE:
        img_batch = np.array(img_batch)
        with gtimer.block("inference"):
            res = inference(img_batch)
        loss = calc_loss(label_batch, res)
        res_list = res_list + list(res.numpy()>0.5)
        label_list = label_list + label_batch
        loss_list.append(loss.numpy())
        img_batch = []
        label_batch = [] 
    if i_step%LOG_STEP==0:
        print("test step - {}/{}        ".format(i_step, N_test), end = '\r')

res_list = np.array(res_list)[:5000,1]
label_list = np.array(label_list)[:5000,1]
loss_list = np.array(loss_list)[:5000]

acc = np.mean(np.equal(res_list, label_list)) * 100
mean_loss = np.mean(loss_list)

print("")
print("=================================================================")
print(
    f'Test Loss: {mean_loss} \n'
    f'Test Accuracy: {acc} \n'
    f'TP / FN / ACC: {np.sum(np.logical_and(res_list, label_list))}, ' 
    f'{np.sum(np.logical_and(np.logical_not(res_list), label_list))}, ' 
    f'{round(np.mean(res_list[np.where(label_list)])*100,2)}\n'
    f'FP / TN / ACC: {np.sum(np.logical_and(res_list, np.logical_not(label_list)))}, '
    f'{np.sum(np.logical_and(np.logical_not(res_list), np.logical_not(label_list)))}, '
    f'{round(np.mean(np.logical_not(res_list[np.where(np.logical_not(label_list))]))*100,2)}\n'
    f'PACC / NACC / TACC: {round(np.mean(label_list[np.where(res_list)])*100,2)}, '
    f'{round(np.mean(np.logical_not(label_list[np.where(np.logical_not(res_list))]))*100,2)}, '
    f'{round(np.mean(res_list==label_list)*100,2)}\n'
)
print("=================================================================")
print("")
print(gtimer)
model_epoch_list.append(last_save)
acc_epoch_list.append(acc)
loss_epoch_list.append(mean_loss)

## Test convergence

In [ ]:
last_model = sorted(os.listdir(ROBOT_MODEL_ROOT))[-1]
logpath = os.path.join(ROBOT_MODEL_ROOT, last_model)

model_epoch_list = []
acc_epoch_list = []
loss_epoch_list = []
# last_save = sorted([item for item in os.listdir(logpath) if item.startswith("model")])[-1]
# last_save = 'model_1'
for last_save in sorted([item for item in os.listdir(logpath) if item.startswith("model")], key=lambda x: int(x[6:])):
    model_log_dir = os.path.join(logpath, last_save)

    import tensorflow as tf
    model = tf.keras.models.load_model(model_log_dir)

    @tf.function
    def inference(images):
        # training=False is only needed if there are layers with different
        # behavior during training versus inference (e.g. Dropout).
        predictions = model(images, training=False)
        return predictions

    loss_object = tf.keras.losses.BinaryCrossentropy()

    @tf.function
    def calc_loss(labels, predictions):
        # training=False is only needed if there are layers with different
        # behavior during training versus inference (e.g. Dropout).
        return loss_object(labels, predictions)
    
    BATCH_SIZE = 50
    LOG_STEP = 100
    N_test = len(dataset_test)
    gtimer = GlobalTimer.instance()
    gtimer.reset()

    i_step = 0
    res_list = []
    label_list = []
    loss_list= []
    img_batch = []
    label_batch = [] 
    for data_path in dataset_test:
        i_step += 1
        full_img, label = load_data(data_path)
        img_batch.append(full_img)
        label_batch.append(label)
        if len(img_batch)==BATCH_SIZE:
            with gtimer.block("inference"):
                res = inference(np.array(img_batch))
            loss = calc_loss(label_batch, res)
            res_list = res_list + list(res.numpy()>0.5)
            label_list = label_list + label_batch
            loss_list.append(loss.numpy())
            img_batch = []
            label_batch = [] 
        if i_step%LOG_STEP==0:
            print("test step - {}/{}        ".format(i_step, N_test), end = '\r')

    res_list = np.array(res_list)[:5000,1]
    label_list = np.array(label_list)[:5000,1]
    loss_list = np.array(loss_list)[:5000]
    
    acc = np.mean(np.equal(res_list, label_list)) * 100
    mean_loss = np.mean(loss_list)

    print("")
    print("=================================================================")
    print(
        f'Test Loss: {mean_loss} \n'
        f'Test Accuracy: {acc} \n'
        f'TP / FN / ACC: {np.sum(np.logical_and(res_list, label_list))}, ' 
        f'{np.sum(np.logical_and(np.logical_not(res_list), label_list))}, ' 
        f'{round(np.mean(res_list[np.where(label_list)])*100,2)}\n'
        f'FP / TN / ACC: {np.sum(np.logical_and(res_list, np.logical_not(label_list)))}, '
        f'{np.sum(np.logical_and(np.logical_not(res_list), np.logical_not(label_list)))}, '
        f'{round(np.mean(np.logical_not(res_list[np.where(np.logical_not(label_list))]))*100,2)}\n'
        f'PACC / NACC / TACC: {round(np.mean(label_list[np.where(res_list)])*100,2)}, '
        f'{round(np.mean(np.logical_not(label_list[np.where(np.logical_not(res_list))]))*100,2)}, '
        f'{round(np.mean(res_list==label_list)*100,2)}\n'
    )
    print("=================================================================")
    print("")
    print(gtimer)
    model_epoch_list.append(last_save)
    acc_epoch_list.append(acc)
    loss_epoch_list.append(mean_loss)

In [ ]:
import matplotlib.pyplot as plt
print(model_epoch_list)
plt.figure(figsize=(10,3))
plt.subplot(1,2,1)
plt.plot(acc_epoch_list)
plt.subplot(1,2,2)
plt.plot(loss_epoch_list)

In [ ]:
save_json(ROBOT_TYPE_NAME+".json", {"epoch": np.array(model_epoch_list), "acc": np.array(acc_epoch_list), "loss": np.array(loss_epoch_list)})

In [ ]:
# data_path = dataset_train[0]
# full_data = load_pickle(data_path)
# full_tar_idx = full_data[b'tar']
# full_tool_idx = full_data[b'tool']
# save_json("tool.json", np.array(np.unravel_index(full_tool_idx, shape=FULL_SHAPE)).transpose())
# save_json("tar.json", np.array(np.unravel_index(full_tar_idx, shape=FULL_SHAPE)).transpose())